In [23]:
import pandas as pd
import re
from tqdm import tqdm
import numpy as np
import nltk.data
from pandas_profiling import ProfileReport
import sys
sys.path.append("../src")
import Preprocessing
import fasttext
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from xgboost import XGBRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error

In [2]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
df = pd.read_csv("../resource/data/blogtext.csv", nrows=200_000)

### Filtering

In [3]:
# filter for a mininmal number of letters in a tweet:
df = df[df["text"].str.count(r"[a-zA-Z]") >= 50]
df = df.reset_index(drop=True)

### Feature Engineering

In [4]:
#def findDates(text):
#    try:
#        return len([date for date in\
#                    datefinder.find_dates(text)])
#    except:
#        return 0

In [5]:
def buildFeatures(text):
    text_split = text.split()
    len_text = len(text)
    sentence_split = tokenizer.tokenize(text)
    
    # find the number of urls in the text
    keywords = ["urlLink","http","www"]
    nb_urls = sum((any(keyword in pattern for keyword in keywords))\
               for pattern in text.split())
    # find the number of mails in the text
    nb_mails = len(re.findall(r"([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+|\bmail\b)"\
                      ,text))
    
    # find the number of dates in the text
#    nb_dates = findDates(text)
     
    # find characteristics about the usage of letters, numbers and symbols
    uppercase_ratio = len(re.findall(r'[A-Z]', text))/len_text
    lowercase_ratio = len(re.findall(r'[a-z]', text))/len_text
    number_ratio = len(re.findall(r'[0-9]', text))/len_text
    symbol_ratio = len(re.findall(r'[$-/:-?{-~!"^_`\[\]]', text))/len_text

    # find characteristics about the letters per word
    sentence_len_word = [len(word) for word in text_split]
    avg_letters_per_word = np.mean([len(word) for word in text_split])
    var_letters_per_word = np.var([len(word) for word in text_split])
    unique_words_ratio = len(set(text_split))/len(text_split)

    # find characteristics about the letters per sentence
    sentence_len_list = [len(sentence) for sentence in sentence_split]
    avg_letters_per_sentence = np.mean(sentence_len_list)
    var_letters_per_sentence = np.var(sentence_len_list)
    
    # find characteristics about the words per sentence
    words_per_sentence_len_list = [len(sentence.split()) for sentence in sentence_split]
    avg_words_per_sentence = np.mean(words_per_sentence_len_list)
    var_words_per_sentence = np.var(words_per_sentence_len_list)
    
    # find the trumps
    uppercase_per_sentence_ratio = [len(re.findall(r'[A-Z]', sentence))/len(sentence)\
                                    for sentence in sentence_split]
    max_sentence_uppercase_ratio = max(uppercase_per_sentence_ratio)
    max_sentence_uppercase_len = len(sentence_split[uppercase_per_sentence_ratio.index(max_sentence_uppercase_ratio)])
    
    return len_text, nb_urls, nb_mails,\
           uppercase_ratio, lowercase_ratio, number_ratio, symbol_ratio,\
           avg_letters_per_word, var_letters_per_word, unique_words_ratio,\
           avg_letters_per_sentence, var_letters_per_sentence,\
           avg_words_per_sentence, var_words_per_sentence,\
           max_sentence_uppercase_ratio, max_sentence_uppercase_len        
           

In [6]:
# append the data
features = [buildFeatures(text) for text  in tqdm(df["text"])]

# append the data
columns = ["Text length", "Number URLs", "Number mails",\
          "Uppercase ratio", "Lowercase ratio", "Number ratio", "Symbol ratio",\
          "Average letters per word", "Variance of letters per word", "Unique words ratio",\
          "Average letters per sentence", "Variance of letters per sentence",\
          "Average words per sentence", "Variance of words per sentence",\
          "Maximal uppercase ratio per sentence", "Length of the maximal uppercase ratio sentence"]

# merge the features with the original dataset
df_preprocessed = df.merge(pd.DataFrame(features, columns=columns), left_index=True, right_index=True)

100%|██████████| 197080/197080 [04:00<00:00, 820.34it/s] 


### Text Preprocessing

In [7]:
# use the preprocessing  module
preprocessor = Preprocessing.Preprocessing()
df_preprocessed["text_preprocessed"] = preprocessor.ProcessMany(df_preprocessed["text"])

# predict the main language
model = fasttext.load_model('../src/data/lid.176.ftz')
df_preprocessed["main_language"] = [model.predict(text)[0][0].split("__")[-1] for text in tqdm(df_preprocessed["text_preprocessed"])]

100%|██████████| 197080/197080 [00:30<00:00, 6486.58it/s]


In [231]:
# drop unnecassary features
df_filtered = df_preprocessed[~(df_preprocessed["main_language"] == "en")]\
                .drop(["id","text","date","main_language"], axis= 1)

### Clustering

### Transformation

In [232]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

class StackedTransformation:
    
    def  __init__(self, X=None, y=None, numerical_transformer=None, text_transformer=None, text_features=None):
        self.X = X
        self.y = y
        self.numerical_transformer = numerical_transformer
        self.text_transformer = text_transformer
        self.text_features = text_features
        
    def data_split(self, X, y, test_size= 0.2):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=test_size, random_state=42)
        return self.X_train, self.X_test, self.y_train, self.y_test
        
    def build_transformer(self, X=None, y=None,\
                          numerical_transformer=None, text_transformer=None, text_features=None):
        # check if variables are present
        if X == None:
            X = self.X
        if y == None:
            y = self.y
        if numerical_transformer == None:
            numerical_transformer = self.numerical_transformer
        if text_transformer == None:
            text_transformer = self.text_transformer
        if text_features == None:
            text_features = self.text_features

        # split the data
        
        self.data_split(X, y)
        
        # Create datasets for each classifier
        self.X_train_text = self.X_train[text_features]
        self.X_test_text =  self.X_test[text_features]
        
        self.X_train_numerical = self.X_train.drop(text_features, axis=1)
        self.X_test_numerical = self.X_test.drop(text_features, axis=1)

        # create transformers 
        self.numerical_transformer = numerical_transformer
        self.text_transformer = text_transformer
        
        #create transformed training batches
        self.X_train_numerical_transformed = self.numerical_transformer.fit_transform(self.X_train_numerical)
        self.X_test_numerical_transformed = self.numerical_transformer.transform(self.X_test_numerical)

        self.X_train_text_transformed = self.text_transformer.fit_transform(self.X_train_text)
        self.X_test_text_transformed = self.text_transformer.transform(self.X_test_text)

    def transform_one(self, text):
        entry = pd.DataFrame(columns=text.keys())
        entry = entry.append(text, ignore_index=True)
        
        # create datasets for each transformer
        entry_text = entry[text_features]
        entry_numerical = entry.drop(text_features, axis=1)
        
        entry_numerical_transformed = self.numerical_transformer.transform(entry_numerical)
        entry_text_transformed = self.text_transformer.transform(entry_text)
        
        return {"transformed_text": entry_text_transformed,\
               "transformed_numerical": entry_numerical_transformed}


In [233]:
X,y = df_filtered.drop("gender", axis=1),df_filtered["gender"]

In [239]:
text_transformer = TfidfVectorizer(ngram_range=(1,1))
numerical_transformer = make_column_transformer((OneHotEncoder(handle_unknown="ignore"), ["age", "topic", "sign"])\
                                                       , remainder=StandardScaler())

text_features = "text_preprocessed"
stacking = StackedTransformation(X, y, numerical_transformer, text_transformer, text_features)
stacking.build_transformer()

### Training 

In [240]:
class StackedModelling:
    
    def __init__(self,  numerical_model, numerical_model_params,\
                text_model, text_model_params, stacked_transformation_instnace):
        
        pass

### Parameter Tuning 

### Model Evaluation 

In [241]:
# test numerical model
X_train_numerical_transformed = stacking.numerical_transformer.fit_transform(stacking.X_train_numerical)
clf = XGBClassifier()
clf.fit(X_train_numerical_transformed, stacking.y_train)
y_pred = clf.predict(stacking.numerical_transformer.transform(stacking.X_test_numerical))

# print mae
print(confusion_matrix(stacking.y_test , y_pred))


[[337  77]
 [112 343]]


In [242]:
# test text model
X_train_text_transformed = stacking.text_transformer.fit_transform(stacking.X_train_text.values)
clf = XGBClassifier()
clf.fit(X_train_text_transformed, stacking.y_train)
y_pred = clf.predict(stacking.text_transformer.transform(stacking.X_test_text))

# print 
print(confusion_matrix(stacking.y_test , y_pred))


[[234 180]
 [135 320]]
